###Install dependencies



In [3]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162995 sha256=a48e00e0e02d738e2b72f26cebd8d80160d58f2c068b5c7d4f55e25f1d33bbad
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [4]:
import pandas as pd
import numpy as np
from surprise import Reader,Dataset
from surprise import KNNBasic
from surprise.model_selection import train_test_split

In [5]:
ratings_df=pd.read_csv('/content/ml-25m/ratings.csv')

In [6]:
ratings_df

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [7]:
del ratings_df['timestamp']   #removing the 'timestamp' column from the ratings_df DataFrame

In [8]:
movies_df = pd.read_csv('/content/ml-25m/movies.csv')

In [9]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [10]:
del movies_df['genres']

In [12]:
merged_df = pd.merge(ratings_df, movies_df, on='movieId') # Merge ratings_df with movies_df on 'movieId' column and store the result in merged_df

In [13]:
merged_df

,userId,movieId,rating,title
0,1,296,5.0,Pulp Fiction (1994)
1,3,296,5.0,Pulp Fiction (1994)
2,4,296,4.0,Pulp Fiction (1994)
3,5,296,4.0,Pulp Fiction (1994)
4,7,296,4.0,Pulp Fiction (1994)
...,...,...,...,...
25000090,162358,200192,2.0,Den frusna leoparden (1986)
25000091,162358,200194,2.0,Tough Luck (2004)
25000092,162386,139970,3.5,I Don't Speak English (1995)
25000093,162386,200726,4.0,The Graduates (1995)


In [28]:
final_df = ratings_df[:20000]  #Sample 20,000 random rows from the ratings_df DataFrame

In [15]:
final_df

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5
...,...,...,...
49995,406,99007,4.5
49996,406,99728,0.5
49997,406,101577,4.5
49998,406,101962,5.0


#### Create a user-item matrix using pivot_table


In [29]:
user_item_matrix =final_df.pivot_table(index='userId', columns='movieId', values='rating')

In [30]:
user_item_matrix

movieId,1,2,3,4,5,6,7,9,10,11,...,194448,194728,195159,195777,195921,200818,200838,203375,203519,205106
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,5.0,3.0,4.5,2.5,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Fill missing values in the user-item matrix with 0

In [31]:
user_item_matrix_filled = user_item_matrix.fillna(0)

In [20]:
user_item_matrix_filled

movieId,1,2,3,4,5,6,7,9,10,11,...,203218,203244,203375,203513,203519,203649,204542,204692,204698,205106
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,2.5,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity_matrix = cosine_similarity(user_item_matrix_filled)

In [39]:
# Assuming cosine_similarity_array is the output array
cosine_similarity_matrix = np.reshape(cosine_similarity_matrix, (len(cosine_similarity_matrix), -1))


In [40]:
cosine_similarity_df = pd.DataFrame(cosine_similarity_matrix)

In [41]:
cosine_similarity_df

,0,1,2,3,4,5,6,7,8,9,...,156,157,158,159,160,161,162,163,164,165
0,1.000000,0.040863,0.061306,0.040815,0.015609,0.000000,0.093852,0.021359,0.022860,0.026149,...,0.000000,0.020598,0.048687,0.086124,0.018969,0.011976,0.000000,0.099245,0.014225,0.021616
1,0.040863,1.000000,0.179009,0.197496,0.158202,0.129720,0.064954,0.176880,0.128820,0.156814,...,0.076186,0.134480,0.268797,0.306876,0.149104,0.142554,0.213079,0.185330,0.089631,0.152910
2,0.061306,0.179009,1.000000,0.357750,0.061448,0.115148,0.031427,0.081003,0.061602,0.132361,...,0.048479,0.068128,0.269388,0.214511,0.077011,0.039412,0.097259,0.206807,0.035703,0.058407
3,0.040815,0.197496,0.357750,1.000000,0.065825,0.072365,0.015566,0.088927,0.066428,0.083821,...,0.035937,0.025046,0.169487,0.199310,0.038850,0.051755,0.140428,0.091286,0.012436,0.058966
4,0.015609,0.158202,0.061448,0.065825,1.000000,0.114936,0.202348,0.307769,0.216435,0.269897,...,0.026422,0.309920,0.066700,0.116266,0.091688,0.087165,0.093528,0.010769,0.326931,0.244873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,0.011976,0.142554,0.039412,0.051755,0.087165,0.041561,0.060765,0.093372,0.135107,0.038399,...,0.000000,0.109656,0.077314,0.093825,0.081486,1.000000,0.036243,0.016526,0.059136,0.133019
162,0.000000,0.213079,0.097259,0.140428,0.093528,0.147805,0.023710,0.142987,0.106539,0.090036,...,0.000000,0.115193,0.129416,0.308197,0.044748,0.036243,1.000000,0.075037,0.078061,0.114208
163,0.099245,0.185330,0.206807,0.091286,0.010769,0.000000,0.000000,0.000000,0.000000,0.025659,...,0.036230,0.000000,0.238609,0.197846,0.057722,0.016526,0.075037,1.000000,0.000000,0.000000
164,0.014225,0.089631,0.035703,0.012436,0.326931,0.047514,0.205459,0.300517,0.198720,0.192458,...,0.016053,0.489652,0.108238,0.090909,0.020074,0.059136,0.078061,0.000000,1.000000,0.278133


In [75]:
reader = Reader()  # Assuming ratings are on a scale from 1 to 5
data = Dataset.load_from_df(cosine_similarity_df.unstack().reset_index(),reader)

###Split the data into training and testing sets using a test size of 25%

In [43]:
trainset, testset = train_test_split(data, test_size=0.25)

###  Initialize and Train a knn model

In [76]:
# Define similarity options
sim_options = {"name": "cosine", "user_based": False}

# Initialize KNNBasic model
knn = KNNBasic(sim_options=sim_options)

In [77]:
# Train the model on the training set
knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [78]:
user_id = 2
top_n_recommendations = knn.get_neighbors(trainset.to_inner_uid(user_id), k=10)

# Print top N recommendations
print("Top 10 Recommendations for User", user_id, ": ", top_n_recommendations)

Top 10 Recommendations for User 2 :  [135, 121, 107, 2, 53, 92, 134, 39, 74, 147]


In [70]:
def get_movie_name(movie_id):
    movie_row = merged_df.loc[merged_df['movieId'] == movie_id]
    if not movie_row.empty:
        return movie_row.iloc[0]['title']
    else:
        return "Unknown Movie"

# Print top N recommendations with movie names
print("Top 10 Recommendations for User", user_id, ": ")
for item_id in top_n_recommendations:
    movie_name = get_movie_name(item_id)
    print(movie_name)

Top 10 Recommendations for User 2 : 
Down Periscope (1996)
Boys of St. Vincent, The (1992)
Muppet Treasure Island (1996)
Jumanji (1995)
Lamerica (1994)
Mary Reilly (1996)
Sonic Outlaws (1995)
Clueless (1995)
Bed of Roses (1996)
Basketball Diaries, The (1995)


In [71]:
predictions = knn.test(testset)

In [72]:
from surprise import accuracy

# Calculate RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)

RMSE: 0.9072
